In [2]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
import requests
import os

In [4]:
api_key = os.getenv("API_KEY")

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {api_key}"
}


In [5]:
CONVERSATION_ID = '67390fee6794dd8f95acdcc3'

In [6]:
conversation_messages_links = f'/v1/conversations/{CONVERSATION_ID}/messages'

In [7]:
from time import sleep

base_url = "https://api.prod2.kustomerapp.com"

url = f"{base_url}{conversation_messages_links}"

response = requests.get(url, headers=headers)

In [8]:
message_data = response.json()

In [9]:
messages = message_data['data']

In [10]:
messages

[{'type': 'message',
  'id': '67390fef79581469e7328a0c',
  'attributes': {'channel': 'chat',
   'app': 'chat',
   'size': 57,
   'direction': 'out',
   'directionType': 'initial-out',
   'preview': 'Olá! Bem vindo ao Ajudaki, o assistente virtual da Daki!\n',
   'meta': {'trackingId': '6737dc15857aab7cf5eec53a'},
   'status': 'sent',
   'assignedTeams': [],
   'assignedUsers': ['66fe99f17533eb52e38851f8'],
   'auto': True,
   'sentAt': '2024-11-16T21:34:39.157Z',
   'createdAt': '2024-11-16T21:34:39.597Z',
   'updatedAt': '2024-11-16T21:34:39.597Z',
   'redacted': False,
   'createdByTeams': [],
   'lang': 'pt_br',
   'rev': 1,
   'reactions': [],
   'firstDelivered': {'timestamp': '2024-11-16T21:34:39.157Z',
    'clientType': 'ios',
    'clientVersion': '4.1.5'},
   'firstRead': {'timestamp': '2024-11-16T21:34:39.157Z'},
   'intentDetections': []},
  'relationships': {'org': {'links': {'self': '/v1/orgs/62c6f23b4a958c4c0ace6ea9'},
    'data': {'type': 'org', 'id': '62c6f23b4a958c4c0ac

In [11]:
import re

def parse_messages(messages):
    """
    Parse a list of message dictionaries and extract:
      1. direction (in/out)
      2. timestamp (sentAt)
      3. content (preview)
    
    Any email addresses or URLs found in the content are redacted.
    
    Args:
        messages (list): List of message dictionaries.
    
    Returns:
        list: List of dictionaries with keys 'direction', 'timestamp', and 'content'.
    """
    # Regular expression to detect email addresses
    email_pattern = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    # Regular expression to detect URLs (http and https)
    url_pattern = re.compile(r'https?://\S+')

    parsed = []
    for msg in messages:
        # Safely extract values from the nested dictionary
        attributes = msg.get('attributes', {})
        direction = attributes.get('direction', 'unknown')
        timestamp = attributes.get('sentAt', None)
        content = attributes.get('preview', '')
        
        # Redact email addresses
        redacted_content = email_pattern.sub('[EMAIL REDACTED]', content)
        # Redact URLs
        redacted_content = url_pattern.sub('[LINK REDACTED]', redacted_content)
        
        parsed.append({
            'direction': direction,
            'timestamp': timestamp,
            'content': redacted_content
        })
    
    return parsed


parsed_messages = parse_messages(messages)
for msg in parsed_messages:
    print(msg)


{'direction': 'out', 'timestamp': '2024-11-16T21:34:39.157Z', 'content': 'Olá! Bem vindo ao Ajudaki, o assistente virtual da Daki!\n'}
{'direction': 'out', 'timestamp': '2024-11-16T21:34:39.158Z', 'content': 'Importante: devido ao alto volume de chamados, nosso tempo de resposta no momento está maior que o habitual.\n\nEscolha entre as opções abaixo como podemos ajudar você e a seguir por favor confirme o seu nome:  '}
{'direction': 'out', 'timestamp': '2024-11-16T21:34:39.159Z', 'content': '**Atenção:** Devido às fortes chuvas, nossos prazos de resposta e entrega podem ser impactados. Estamos trabalhando para minimizar qualquer atraso e agradecemos a sua compreensão. '}
{'direction': 'out', 'timestamp': '2024-11-16T21:34:39.160Z', 'content': 'Como podemos te ajudar hoje?'}
{'direction': 'in', 'timestamp': '2024-11-16T21:34:39.161Z', 'content': '🚲 Ajuda com um Pedido '}
{'direction': 'out', 'timestamp': '2024-11-16T21:34:40.124Z', 'content': 'Por favor, selecione o Pedido sobre o qual 

In [12]:
from datetime import datetime

def sort_parsed_messages_by_timestamp(parsed_messages, reverse=False):
    """
    Sorts the parsed messages by the 'timestamp' key.
    
    Args:
        parsed_messages (list): List of dictionaries containing message info with a 'timestamp' key.
        reverse (bool): If True, sorts in descending order. Defaults to False (ascending).
    
    Returns:
        list: The sorted list of messages.
    """
    # Convert the ISO timestamp to a datetime object for proper sorting
    sorted_messages = sorted(
        parsed_messages, 
        key=lambda msg: datetime.fromisoformat(msg["timestamp"].replace("Z", "+00:00")),
        reverse=reverse
    )
    return sorted_messages
    
sorted_messages = sort_parsed_messages_by_timestamp(parsed_messages)
for message in sorted_messages:
    print(message)

{'direction': 'out', 'timestamp': '2024-11-16T21:34:39.157Z', 'content': 'Olá! Bem vindo ao Ajudaki, o assistente virtual da Daki!\n'}
{'direction': 'out', 'timestamp': '2024-11-16T21:34:39.158Z', 'content': 'Importante: devido ao alto volume de chamados, nosso tempo de resposta no momento está maior que o habitual.\n\nEscolha entre as opções abaixo como podemos ajudar você e a seguir por favor confirme o seu nome:  '}
{'direction': 'out', 'timestamp': '2024-11-16T21:34:39.159Z', 'content': '**Atenção:** Devido às fortes chuvas, nossos prazos de resposta e entrega podem ser impactados. Estamos trabalhando para minimizar qualquer atraso e agradecemos a sua compreensão. '}
{'direction': 'out', 'timestamp': '2024-11-16T21:34:39.160Z', 'content': 'Como podemos te ajudar hoje?'}
{'direction': 'in', 'timestamp': '2024-11-16T21:34:39.161Z', 'content': '🚲 Ajuda com um Pedido '}
{'direction': 'out', 'timestamp': '2024-11-16T21:34:40.124Z', 'content': 'Por favor, selecione o Pedido sobre o qual 